# Приоритизация гипотез и анализ A/B теста

## Описание проекта:
В данном проекте по заданию отдела маркетинга проводится приоритизация гипотез по фреймворкам ICE и RICE. Далее комплексно анализируются результаты A/B теста, который был запущен в течение одного месяца, по различным кумулятивным метрикам на сырых и отфильтрованных данных. Полученные результаты позволили выявить наличие/отсутствие различий между исследуемыми группами и могут считаться ценными для бизнеса.

## Главные выводы:

* Конверсия в группе B выше, чем в группе A - на сырых данных на 13.8%, а на чистых на 16.8%. Различие в конверсии между группами является статистически значимым и на сырых, и на чистых данных. После удаления аномалий - p-value немного снизился.


* Различие в среднем чеке группы B выше группы A на 25.9% на сырых данных, а на чистых ниже на 1.6%. Ни на сырых, ни на чистых данных, различие в среднем чеке не является статистически значимым. После фильтрации аномалий p-value вырос по сравнению с сырыми данными.


* График относительного изменения кумулятивной конверсии демонстрирует явное превосходство группы B, преимущество которой стабилизировалось на уровне 13.8%.


* График относительного изменения кумулятивного среднего чека демонстрирует резкие колебания на сырых данных, однако после фильтрации аномалий, различия среднего чека сократились до 1.6%. При этом различие нельзя считать статистически значимым.


**Итог:** Таким образом, мы можем заключить, что A/B тест был проведен успешно. Было зафиксировано, что конверсия группы B существенно выше, чем в группе A, и этот результат статистически значим. Также мы выяснили, что значимые различия в среднем чеке между группами отсутствуют. Тест может быть остановлен.

## Содержание проекта: <a name="introduction"></a>

### 1. [Приоритизация гипотез](#paragraph1)

### 2. [Анализ A/B теста](#paragraph2)

#### 2.1 [Обзор и предобработка данных](#subparagraph2_1)

* 2.1.1 [Таблица orders - информация о заказах](#subparagraph2_1_1)
* 2.1.2 [Таблица visitors - информация о посетителях](#subparagraph2_1_2)

#### 2.2 [Исследовательский анализ данных](#subparagraph2_2)

* 2.2.1 [График кумулятивной выручки по группам](#subparagraph2_2_1)
* 2.2.2 [График кумулятивного среднего чека по группам](#subparagraph2_2_2)
* 2.2.3 [График относительного различия кумулятивного среднего чека между группами](#subparagraph2_2_3)
* 2.2.4 [График кумулятивной конверсии по группам](#subparagraph2_2_4)
* 2.2.5 [График относительного изменения кумулятивной конверсии группы B к группе A](#subparagraph2_2_5)
* 2.2.6 [Точечный график количества заказов по пользователям](#subparagraph2_2_6)
* 2.2.7 [Оценка 95 и 99 перцентили количества заказов на пользователя](#subparagraph2_2_7)
* 2.2.8 [Точечный график стоимостей заказов](#subparagraph2_2_8)
* 2.2.9 [Оценка 95 и 99 перцентили стоимостей заказов](#subparagraph2_2_9)

#### 2.3 [Проверка статистических гипотез](#subparagraph2_3)

* 2.3.1 [Статистическая значимость различий в конверсии между группами по сырым данным](#subparagraph2_3_1)
* 2.3.2 [Статистическая значимость различий в среднем чеке между группами по сырым данным](#subparagraph2_3_2)
* 2.3.3 [Статистическая значимость различий в конверсии между группами по чистым данным](#subparagraph2_3_3)
* 2.3.4 [Статистическая значимость различий в среднем чеке между группами по чистым данным](#subparagraph2_3_4)

### 3. [Общий вывод](#paragraph3)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from scipy import stats as st
from math import sqrt
pio.templates.default = 'none'
pio.renderers.default = 'notebook_connected'

## 1. Приоритизация гипотез <a name="paragraph1"></a>

In [2]:
hypothesis = pd.read_csv('hypothesis.csv')
hypothesis.columns = map(str.lower, hypothesis.columns)

In [3]:
hypothesis

,hypothesis,reach,impact,confidence,efforts
0,"Добавить два новых канала привлечения трафика,...",3,10,8,6
1,"Запустить собственную службу доставки, что сок...",2,5,4,10
2,Добавить блоки рекомендаций товаров на сайт ин...,8,3,7,3
3,"Изменить структура категорий, что увеличит кон...",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы уве...",3,1,1,1
5,"Добавить страницу отзывов клиентов о магазине,...",3,2,2,3
6,Показать на главной странице баннеры с актуаль...,5,3,8,3
7,Добавить форму подписки на все основные страни...,10,7,8,5
8,"Запустить акцию, дающую скидку на товар в день...",1,9,9,5


`Hypothesis` — краткое описание гипотезы;
`Reach` — охват пользователей по 10-балльной шкале;
`Impact` — влияние на пользователей по 10-балльной шкале;
`Confidence` — уверенность в гипотезе по 10-балльной шкале;
`Efforts` — затраты ресурсов на проверку гипотезы по 10-балльной шкале.

In [4]:
hypothesis['hypothesis'].unique()

array(['Добавить два новых канала привлечения трафика, что позволит привлекать на 30% больше пользователей',
       'Запустить собственную службу доставки, что сократит срок доставки заказов',
       'Добавить блоки рекомендаций товаров на сайт интернет магазина, чтобы повысить конверсию и средний чек заказа',
       'Изменить структура категорий, что увеличит конверсию, т.к. пользователи быстрее найдут нужный товар',
       'Изменить цвет фона главной страницы, чтобы увеличить вовлеченность пользователей',
       'Добавить страницу отзывов клиентов о магазине, что позволит увеличить количество заказов',
       'Показать на главной странице баннеры с актуальными акциями и распродажами, чтобы увеличить конверсию',
       'Добавить форму подписки на все основные страницы, чтобы собрать базу клиентов для email-рассылок',
       'Запустить акцию, дающую скидку на товар в день рождения'],
      dtype=object)

* Фреймворк ICE

$$ICE = \frac{Impact * Confidence}{Effort}$$

In [5]:
hypothesis['ice'] = (hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']

In [6]:
hypothesis[['hypothesis', 'ice']].sort_values(by='ice', ascending=False)

,hypothesis,ice
8,"Запустить акцию, дающую скидку на товар в день...",16.200000
0,"Добавить два новых канала привлечения трафика,...",13.333333
7,Добавить форму подписки на все основные страни...,11.200000
6,Показать на главной странице баннеры с актуаль...,8.000000
2,Добавить блоки рекомендаций товаров на сайт ин...,7.000000
1,"Запустить собственную службу доставки, что сок...",2.000000
5,"Добавить страницу отзывов клиентов о магазине,...",1.333333
3,"Изменить структура категорий, что увеличит кон...",1.125000
4,"Изменить цвет фона главной страницы, чтобы уве...",1.000000


* Фреймворк RICE

$$RICE = \frac{Reach * Impact * Confidence}{Effort}$$

In [7]:
hypothesis['rice'] = (hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']

In [8]:
hypothesis[['hypothesis', 'rice']].sort_values(by='rice', ascending=False)

,hypothesis,rice
7,Добавить форму подписки на все основные страни...,112.0
2,Добавить блоки рекомендаций товаров на сайт ин...,56.0
0,"Добавить два новых канала привлечения трафика,...",40.0
6,Показать на главной странице баннеры с актуаль...,40.0
8,"Запустить акцию, дающую скидку на товар в день...",16.2
3,"Изменить структура категорий, что увеличит кон...",9.0
1,"Запустить собственную службу доставки, что сок...",4.0
5,"Добавить страницу отзывов клиентов о магазине,...",4.0
4,"Изменить цвет фона главной страницы, чтобы уве...",3.0


**Вывод:** мы провели приоритизацию списка гипотез по 2 методологиям: **ICE** и **RICE**. При включении параметра `reach` список значительно изменился:
* *Акция в день рождения* опустилась с первого на пятое место - день рождения в ближайшем будущем будет лишь у небольшой доли клиентов, соответственно охват гипотезы не так велик. 
* *Новые каналы привлечения трафика* незначительно опустились - со второго на третье место, но эта гипотеза по-прежнему осталась в топе. 
* *Добавление формы подписки на все основные страницы* поднялось с третьего на первое место - эта гипотеза имеет максимальный охват, потому что затронет всех пользователей. 
* *Показ на главной странице баннеров с актуальными акциями* остался на четвертом месте без изменений.
* *Добавление блоков рекомендаций товаров* поднялось с пятого на второе место - такие масштабные изменения увидят все пользователи сайта.
* *Собственная служба доставки* незначительно опустилась с шестого на седьмое место.
* *Страница отзывов* опустилась с седьмого на восьмое место.
* *Изменение структуры категорий* поднялось с восьмого на шестое место - существенные изменения сайта опять же повлияют на большинство пользователей.
* *Изменение цвета фона главной страницы* осталось на последнем (девятом) месте без изменений.

[Вернуться к оглавлению](#introduction)

## 2. Анализ A/B теста <a name="paragraph2"></a>

### 2.1 Обзор и предобработка данных<a name="subparagraph2_1"></a>

In [9]:
orders = pd.read_csv('orders.csv')
visitors = pd.read_csv('visitors.csv')

#### 2.1.1 Таблица `orders` - информация о заказах<a name="subparagraph2_1_1"></a>

In [10]:
orders.columns = map(str.lower, orders.columns)

In [11]:
orders

,transactionid,visitorid,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,6490,B
1193,2203539145,370388673,2019-08-14,3190,A
1194,1807773912,573423106,2019-08-14,10550,A
1195,1947021204,1614305549,2019-08-14,100,A


In [12]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionid  1197 non-null   int64 
 1   visitorid      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


`transactionid` — идентификатор заказа, `visitorid` — идентификатор пользователя, совершившего заказ, `date` — дата, когда был совершён заказ, `revenue` — выручка заказа, `group` — группа A/B-теста, в которую попал заказ.

Пропуски в таблице `orders` отсутствуют.

Приведем столбец `date` к формату datetime.

In [13]:
orders['date'] = pd.to_datetime(orders['date'], format='%Y-%m-%d')

#### 2.1.2 Таблица `visitors` - информация о посетителях<a name="subparagraph2_1_2"></a>

In [14]:
visitors

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [15]:
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


`date` — дата, `group` — группа A/B-теста, `visitors` — количество пользователей в указанную дату в указанной группе A/B-теста.

Пропуски в таблице `visitors` отсутствуют.

Приведем столбец `date` к формату datetime.

In [16]:
visitors['date'] = pd.to_datetime(visitors['date'], format='%Y-%m-%d')

### 2.2 Исследовательский анализ данных<a name="subparagraph2_2"></a>

#### 2.2.1 График кумулятивной выручки по группам<a name="subparagraph2_2_1"></a>

Cоздаем массив уникальных пар значений дат и групп теста `dates_groups`.

In [17]:
dates_groups = orders[['date', 'group']].drop_duplicates()

Получаем агрегированные по дням кумулятивные данные о заказах `orders_agg`.

In [18]:
orders_agg = dates_groups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]\
.agg({'date' : 'max', 'group' : 'max', 'transactionid' : 'nunique', 'visitorid' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

Получаем агрегированные по дням кумулятивные данные о посетителях интернет-магазина `visitors_agg`.

In [19]:
visitors_agg = dates_groups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group']) 

Объединяем кумулятивные данные в одной таблице `cumulative_data`.

In [20]:
cumulative_data = orders_agg.merge(visitors_agg, left_on=['date', 'group'], right_on=['date', 'group'])
cumulative_data.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

Создадим датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А `cumulative_revenue_A`.

In [21]:
cumulative_revenue_A = cumulative_data.query('group == "A"')[['date','revenue', 'orders']]

Создадим датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B `cumulative_revenue_B`.

In [22]:
cumulative_revenue_B = cumulative_data.query('group == "B"')[['date','revenue', 'orders']]

Построим график кумулятивной выручки.

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_revenue_A['date'], y=cumulative_revenue_A['revenue'],
                    mode='lines',
                    name='Group A'))
fig.add_trace(go.Scatter(x=cumulative_revenue_B['date'], y=cumulative_revenue_B['revenue'],
                    mode='lines',
                    name='Group B'))
fig.update_layout(title="Кумулятивная выручка по группам",
    xaxis_title="Дата",
    yaxis_title="Выручка в рублях",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show();

**Вывод:** накопленная выручка каждой из групп имела устойчивый восходящий тренд в период проведения теста. *Группа B* превосходила *группу A* на протяжении всего исследуемого периода за исключением одного дня - 1 августа 2019. 5 и 13 августа разрыв между группами был минимальным с незначительным преимуществом *группы B*, и каждый раз *группе B* удавалось усилить свое лидерство. С 18 по 19 августа 2019 выручка *группы B* резко увеличилась в 1.5 раза (с 2,7 млн до 4.1 млн руб.) - это может говорить о том, что в *группе B* появились аномально дорогие заказы или их нетипично большое  количество за короткое время. *Группе A* не удалось сократить отставание на конец исследуемого периода. 

#### 2.2.2 График кумулятивного среднего чека по группам<a name="subparagraph2_2_2"></a>

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_revenue_A['date'], y=cumulative_revenue_A['revenue']/cumulative_revenue_A['orders'],
                    mode='lines',
                    name='Group A'))
fig.add_trace(go.Scatter(x=cumulative_revenue_B['date'], y=cumulative_revenue_B['revenue']/cumulative_revenue_B['orders'],
                    mode='lines',
                    name='Group B'))
fig.update_layout(title="Кумулятивный средний чек по группам",
    xaxis_title="Дата",
    yaxis_title="Средний чек в рублях",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show();

**Вывод:** на начало исследуемого периода (1 августа) *группа А* имела более высокий средний чек, однако уже на следующий день уступила *группе B*. К 12 августа *группа A* ненадолго восстановила лидерство над *группой B*, достигнув собственного пика 13 августа. С 16 августа накопленный средний чек стал выше в *группе B* и ее преимущество сохранилось до конца теста, несмотря на медленное сокращение разрыва между группами, начиная с 19 августа. График накопленного среднего чека также демонстрирует, что в *группе B* присутствовал некий выброс 18-19 августа, который закрепил ее преимущество и мог повлиять на результаты теста. Одной из причин, как мы уже упоминали ранее, могли быть некоторые аномально крупные заказы.

#### 2.2.3 График относительного различия кумулятивного среднего чека между группами<a name="subparagraph2_2_3"></a>

Соединим данные по кумулятивным выручкам в `cumulative_revenue`.

In [25]:
cumulative_revenue = cumulative_revenue_A.merge(cumulative_revenue_B, left_on='date', right_on='date', how='left', suffixes=['_A', '_B'])

Рассчитаем отношения средних чеков между группами `average_purchase_ratio_BA`.

In [26]:
average_purchase_ratio_BA = ((cumulative_revenue['revenue_B'] / cumulative_revenue['orders_B']) / (cumulative_revenue['revenue_A'] / cumulative_revenue['orders_A'])) - 1

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_revenue['date'], y=average_purchase_ratio_BA,
                    mode='lines',
                    name='Отношение группы B к A'))
fig.update_layout(title="Относительное изменение кумулятивного среднего чека группы B к A",
    xaxis_title="Дата",
    yaxis_title="Отношение среднего чека между группами",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show();

**Вывод:** график отношения накопленных средних чеков между группами позволяет более явно увидеть, что *группа B* имела преимущество над *группой A* в течение практически всего времени проведения теста. Стабилизация наступила к концу исследуемого периода на уровне около 26%. Кроме того, резкие скачки и падения на графике могут указывать на выбросы в данных.

#### 2.2.4 График кумулятивной конверсии по группам<a name="subparagraph2_2_4"></a>

Расчитаем кумулятивную конверсию `conversion`.

In [28]:
cumulative_data['conversion'] = cumulative_data['orders'] / cumulative_data['visitors']

Кумулятивная конверсия группы A

In [29]:
cumulative_data_A = cumulative_data.query('group == "A"')

Кумулятивная конверсия группы B

In [30]:
cumulative_data_B = cumulative_data.query('group == "B"')

Построим график кумулятивной конверсии по группам.

In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_data_A['date'], y=cumulative_data_A['conversion'],
                    mode='lines',
                    name='Group A'))
fig.add_trace(go.Scatter(x=cumulative_data_B['date'], y=cumulative_data_B['conversion'],
                    mode='lines',
                    name='Group B'))
fig.update_layout(title="Кумулятивная конверсия по группам",
    xaxis_title="Дата",
    yaxis_title="Конверсия",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99),
    autosize=False,
    width=1000,
    height=650)
fig.show();

**Вывод:** конверсия *группы A* резко увеличилась в начале теста (3 августа), однако уже с 6 августа до конца теста *группа B* вырвалась вперед и сохранила преимущество до конца теста. Конверсия *группы A* в свою очередь снизилась и также зафиксировалась.

#### 2.2.5 График относительного изменения кумулятивной конверсии группы B к группе A<a name="subparagraph2_2_5"></a>

In [32]:
сumulative_сonversion = cumulative_data_A[['date','conversion']].merge(cumulative_data_B[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['_A', '_B'])

In [33]:
cumulative_conversion_ratio_BA = (сumulative_сonversion['conversion_B'] / сumulative_сonversion['conversion_A']) - 1

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=сumulative_сonversion['date'], y=cumulative_conversion_ratio_BA,
                    mode='lines',
                    name='Отношение группы B к A'))
fig.update_layout(title="Относительное изменение кумулятивной конверсии группы B к A",
    xaxis_title="Дата",
    yaxis_title="Отношение конверсии между группами",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show();

**Вывод:** в начале теста присутствовали некоторые флуктуации, однако на протяжении почти всего времени проведения теста конверсия в *группе B* была выше, стабилизировавшись на уровне около 14% выше уровня конверсии *группы A*.

#### 2.2.6 Точечный график количества заказов по пользователям<a name="subparagraph2_2_6"></a>

In [35]:
orders_by_visitors = orders.groupby('visitorid')['transactionid'].count().reset_index()

In [36]:
fig = go.Figure(data=go.Scatter(x=orders_by_visitors.index, y=orders_by_visitors['transactionid'], mode='markers'))
fig.update_layout(title="Точечный график количества заказов по пользователям",
    xaxis_title="Порядковый номер клиента",
    yaxis_title="Количество заказов",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show()

**Вывод:** абсолютное большинство пользователей сделало один заказ. Кроме того, относительно небольшое число  пользователей сделали 2 заказа. Более 3 заказов уже можно считать довольно редким событием. Также видим, что два пользователя сделали по 11 заказов, что является экстремально высоким значением.

#### 2.2.7 Оценка 95 и 99 перцентили количества заказов на пользователя<a name="subparagraph2_2_7"></a>

In [37]:
np.percentile(orders_by_visitors['transactionid'], [95, 99])

array([2., 4.])

**Вывод:** 95% пользователей сделали не больше 2 заказов, а 99% пользователей - не больше 4 заказов. Разумно провести границу "нормальности" пользователей по 2 покупкам - таким образом, если пользователь совершает три или более покупок, то его можно считать аномальным.

#### 2.2.8 Точечный график стоимостей заказов<a name="subparagraph2_2_8"></a>

In [38]:
fig = go.Figure(data=go.Scatter(x=orders.index, y=orders['revenue'], mode='markers'))
fig.update_layout(title="Точечный график стоимостей заказов",
    xaxis_title="Порядковый номер заказа",
    yaxis_title="Выручка с заказа",
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01),
    autosize=False,
    width=1000,
    height=650)
fig.show()

**Вывод:** на графике выше можно явно увидеть один экстремально дорогой заказ на 1.3 млн руб. Кроме того, существенно выделяется заказ на 202.7 тыс. руб. Остальные заказы примерно в одном диапазоне по стоимости.

#### 2.2.9 Оценка 95 и 99 перцентили стоимостей заказов<a name="subparagraph2_2_9"></a>

In [39]:
np.percentile(orders['revenue'], [95, 99])

array([28000. , 58233.2])

**Вывод:** Стоимость 95% заказов не превышает 28 тыс. руб., а 99% заказов стоят не дороже 58233.2 руб. Следовательно, логично провести границу нормальной стоимости для анализа на уровне 28 тысяч рублей. Все заказы дороже 28 тысяч будем считать аномальными.

### 2.3 Проверка статистических гипотез<a name="subparagraph2_3"></a>

#### 2.3.1 Статистическая значимость различий в конверсии между группами по сырым данным<a name="subparagraph2_3_1"></a>

In [40]:
orders_by_users_A = orders.query('group == "A"').groupby('visitorid')['transactionid'].count().reset_index()
orders_by_users_B = orders.query('group == "B"').groupby('visitorid')['transactionid'].count().reset_index()

In [41]:
visitors_by_date_A = visitors.query('group == "A"')[['date', 'visitors']].copy()
visitors_by_date_B = visitors.query('group == "B"')[['date', 'visitors']].copy()

In [42]:
conversion_sample_A = pd.concat([orders_by_users_A['transactionid'],pd.Series(0, index=np.arange(visitors_by_date_A['visitors'].sum() - len(orders_by_users_A['transactionid'])), name='orders')],axis=0)
conversion_sample_B = pd.concat([orders_by_users_B['transactionid'],pd.Series(0, index=np.arange(visitors_by_date_B['visitors'].sum() - len(orders_by_users_B['transactionid'])), name='orders')],axis=0)

In [43]:
st.mannwhitneyu(conversion_sample_A, conversion_sample_B)

MannwhitneyuResult(statistic=176473106.0, pvalue=0.008396177528376304)

In [44]:
print("{0:.3f}".format(conversion_sample_B.mean()/conversion_sample_A.mean()-1)) 

0.138


**Вывод:** на сырых данных *группа B* имеет относительное преимущество (конверсия выше на 13.8%). Разница между конверсиями в группах оказалась статистически значимой - p-value значительно меньше уровня значимости 0.05. Нулевая гипотеза об отсутствии различий между группами отвергается. Принимается альтернативная гипотеза о существовании различий между группами.

#### 2.3.2 Статистическая значимость различий в среднем чеке между группами по сырым данным<a name="subparagraph2_3_2"></a>

In [45]:
average_purchase_sample_A = orders.query('group == "A"')['revenue']
average_purchase_sample_B = orders.query('group == "B"')['revenue']

In [46]:
st.mannwhitneyu(average_purchase_sample_A, average_purchase_sample_B)

MannwhitneyuResult(statistic=176175.0, pvalue=0.3646454927716229)

In [47]:
print("{0:.3f}".format(average_purchase_sample_B.mean()/average_purchase_sample_A.mean()-1)) 

0.259


**Вывод:** несмотря на то, что средний чек в *группе B* выше на 25.9%, на сырых данных разница между средними чеками в *группах A и B* оказалась статистически не значимой - с вероятностью 36.46% различие могло быть получено случайно - так как p-value выше 0.05, нет оснований отвергать нулевую гипотезу об отсутствии различий.

#### 2.3.3 Статистическая значимость различий в конверсии между группами по чистым данным<a name="subparagraph2_3_3"></a>

In [48]:
clean_orders_by_users_A = orders_by_users_A.query('transactionid <= 2')
clean_orders_by_users_B = orders_by_users_B.query('transactionid <= 2')

In [49]:
clean_conversion_sample_A = pd.concat([clean_orders_by_users_A['transactionid'],pd.Series(0, index=np.arange(visitors_by_date_A['visitors'].sum() - len(clean_orders_by_users_A['transactionid'])), name='orders')],axis=0)
clean_conversion_sample_B = pd.concat([clean_orders_by_users_B['transactionid'],pd.Series(0, index=np.arange(visitors_by_date_B['visitors'].sum() - len(clean_orders_by_users_B['transactionid'])), name='orders')],axis=0)

In [50]:
st.mannwhitneyu(clean_conversion_sample_A, clean_conversion_sample_B)

MannwhitneyuResult(statistic=176452160.5, pvalue=0.006451895594778342)

In [51]:
print("{0:.3f}".format(clean_conversion_sample_B.mean()/clean_conversion_sample_A.mean()-1)) 

0.168


**Вывод:** на чистых данных без выбросов *группа B* имеет еще чуть большее относительное преимущество, чем на сырых данных (16.8%), кроме того различие между конверсиями в группах оказалось статистически значимым - p-value значительно ниже уровня значимости 0.05 - нулевая гипотеза об отсутствии различий отвергается. Принимается альтернативная гипотеза о существовании значимых различий между конверсиями в разных группах.

#### 2.3.4 Статистическая значимость различий в среднем чеке между группами по чистым данным<a name="subparagraph2_3_4"></a>

In [52]:
clean_average_purchase_sample_A = average_purchase_sample_A.reset_index().query('revenue <= 28000')['revenue']
clean_average_purchase_sample_B = average_purchase_sample_B.reset_index().query('revenue <= 28000')['revenue']

In [53]:
st.mannwhitneyu(clean_average_purchase_sample_A, clean_average_purchase_sample_B)

MannwhitneyuResult(statistic=159861.0, pvalue=0.4259370929702875)

In [54]:
print("{0:.3f}".format(clean_average_purchase_sample_B.mean()/clean_average_purchase_sample_A.mean()-1)) 

-0.016


**Вывод:** на чистых данных преимущество среднего чека *группы B* исчезло - различие стало составлять всего -1.6%, близко к нулю. В то же время p-value увеличилось и осталось существенно выше уровня значимости 0.05 - по-прежнему нет оснований отвергать нулевую гипотезу об отсутствии различий между группами. Иными словами, различия в среднем чеке между группами статистически не значимы.

[Вернуться к оглавлению](#introduction)

## 3. Общий вывод <a name="paragraph3"></a>

* Конверсия в *группе B* выше, чем в *группе A* - на сырых данных на 13.8%, а на чистых на 16.8%. **Различие в конверсии между группами является статистически значимым** и на сырых, и на чистых данных. После удаления аномалий - p-value немного снизился.
* Различие в среднем чеке *группы B* выше *группы A* на 25.9% на сырых данных, а на чистых ниже на 1.6%. Ни на сырых, ни на чистых данных, **различие в среднем чеке не является статистически значимым**. После фильтрации аномалий p-value вырос по сравнению с сырыми данными.
* График относительного изменения кумулятивной конверсии демонстрирует явное превосходство *группы B*, преимущество которой стабилизировалось на уровне 13.8%.
* График относительного изменения кумулятивного среднего чека демонстрирует резкие колебания на сырых данных, однако после фильтрации аномалий, различия среднего чека сократились до 1.6%. При этом, как мы отметили ранее различие нельзя считать статистически значимым.

Таким образом, мы можем заключить, что A/B тест был проведен успешно. Было зафиксировано, что конверсия *группы B* существенно выше, чем в *группе A*, и этот результат статистически значим. Также мы выяснили, что значимые различия в среднем чеке между группами отсутствуют. Полученные результаты являются ценными для бизнеса, тест может быть остановлен.

[Вернуться к оглавлению](#introduction)